In [8]:
#@title Packages and functions
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jv, jvp, yv, iv
from scipy.special import spherical_jn, spherical_yn
from scipy.optimize import root_scalar
plt.rcParams.update({'font.size': 22})
#%matplotlib qt

def Valle_Modes(cL, cT, rho, eta, R, order, modes):
  def Valle_matrix(omega, kh, cL, cT, rho, nu, R):
      '''
      Determinant of coefficients for the 6x6 matrix in the thesis of Christine Valle.
      kh = dimensionless wavenumber (equivalent to kp in Cerv's code)
      b = Outer radius
      a = Inner radius
      cL1 = Longitudinal velocity of medium 1
      cT1 = Transversal velocity of medium 1
      cL2 = Longitudinal velocity of medium 2
      cT2 = Transversal velocity of medium 2
      '''
      # SUBROUTINES NEEDED:
      def lame_coefficients(Vp1, Vs1, den1, Vp2, Vs2, den2):
          '''
          Calculate the Lame coefficients for the two media.
          Vp1, Vs1, den1 = Velocities and densities of medium 1.
          Vp2, Vs2, den2 = Velocities and densities of medium 2.
          Output: Lame coeficients in GPa
          lame_coeffs = [mu1, lambda1, mu2, lambda2]
          '''
          mu1 = den1*Vs1**2
          lambda1 = den1*(Vp1**2 - 2*Vs1**2)
          mu2 = den2*Vs2**2
          lambda2 = den2*(Vp2**2 - 2*Vs2**2)
          lame_coeffs = np.array([mu1, lambda1, mu2, lambda2])
          return lame_coeffs*1e-9

      def mu_over_lambda(lame_coeffs,medium_numerator, medium_denominator):
          '''
          Calculate mu_(medium_numerator)/lambda_(medium_denominator) for a double-layered cylinder.
          Input:
          lame_coeffs = array with the lame coefficients as calculated by function 'lame_coefficients'
          lame_coeffs = [mu1, lambda1, mu2, lambda2]
          medium =  1 (outermost layer. Integer)
          medium =  2 (outermost layer. Integer.)
          '''
          if (medium_numerator == 1) & (medium_denominator == 1):
              #print('mu1/lambda1: ')
              val = lame_coeffs[medium_numerator-1]/lame_coeffs[medium_denominator]
          elif (medium_numerator == 1) & (medium_denominator == 2):
              #print('mu1/lambda2: ')
              val = lame_coeffs[medium_numerator-1]/lame_coeffs[medium_denominator+1]
          elif (medium_numerator == 2) & (medium_denominator == 1):
              #print('mu2/lambda1: ')
              val = lame_coeffs[medium_numerator]/lame_coeffs[medium_denominator]
          return val

      def lambda_over_lambda(lame_coeffs,medium_numerator = 2, medium_denominator = 1):
          '''
          Calculate mu_(medium_numerator)/lambda_(medium_denominator) for a double-layered cylinder.
          Input:
          lame_coeffs = array with the lame coefficients as calculated by function 'lame_coefficients'
          lame_coeffs = [mu1, lambda1, mu2, lambda2]
          medium =  1 (outermost layer. Integer)
          medium =  2 (outermost layer. Integer.)
          '''
          val = lame_coeffs[medium_numerator+1]/lame_coeffs[medium_denominator]
          return val

      # CODE BEGINS:
      cL1 = cL[0]
      cL2 = cL[1]
      cT1 = cT[0]
      cT2 = cT[1]
      rho1 = rho[0]
      rho2 = rho[1]
      # Calculate the Lame coefficients (lambda and mu)
      lame_coeffs = lame_coefficients(cL1, cT1, rho1, cL2, cT2, rho2)
      # re-scaling for numerical convenience
      R = R*1000
      cL1 = cL1/1000
      cL2 = cL2/1000
      cT1 = cT1/1000
      cT2 = cT2/1000
      omega = omega/1e6
      #omega_m1 = omega*h/cT1
      #k_m = k*h
      h = R*(1 - nu)
      #nu = a/b
      omega_h1 = omega*h/(cT1*(1-nu)) # omega*b/cT1 #
      omega_h2 = omega*h/(cT2*(1-nu)) # omega*b/cT2 #
      kappa1 = cL1/cT1
      kappa2 = cL2/cT2
      # The coefficients of the matrix
      # 1st row ################################################################################################################
      d11 = (2*mu_over_lambda(lame_coeffs,1, 1)*kh*(kh-1) - (1 + 2*mu_over_lambda(lame_coeffs,1, 1))*(omega_h1/kappa1)**2)*jv(kh,omega_h1/kappa1) + 2*(omega_h1/kappa1)*mu_over_lambda(lame_coeffs,1, 1)*jv(kh+1,omega_h1/kappa1)
      d12 = (2*mu_over_lambda(lame_coeffs,1, 1)*kh*(kh-1) - (1 + 2*mu_over_lambda(lame_coeffs,1, 1))*(kh/kappa1)**2)*yv(kh,omega_h1/kappa1) + 2*(omega_h1/kappa1)*mu_over_lambda(lame_coeffs,1, 1)*yv(kh+1,omega_h1/kappa1)
      d13 = 2*mu_over_lambda(lame_coeffs,1, 1)*1j*kh*((kh-1)*jv(kh,omega_h1) - omega_h1*jv(kh+1,omega_h1))
      d14 = 2*mu_over_lambda(lame_coeffs,1, 1)*1j*kh*((kh-1)*yv(kh,omega_h1) - omega_h1*yv(kh+1,omega_h1))
      d15 = 0.0; d16 = 0.0;
      # 2nd row ################################################################################################################
      d21 = 2*1j*kh*((kh - 1)*jv(kh,omega_h1/kappa1) - (omega_h1/kappa1)*jv(kh+1,omega_h1/kappa1))
      d22 = 2*1j*kh*((kh - 1)*yv(kh,omega_h1/kappa1) - (omega_h1/kappa1)*yv(kh+1,omega_h1/kappa1))
      d23 = (omega_h1**2 - 2*kh*(kh - 1))*jv(kh,omega_h1) - 2*omega_h1*jv(kh+1,omega_h1)
      d24 = (omega_h1**2 - 2*kh*(kh - 1))*yv(kh,omega_h1) - 2*omega_h1*yv(kh+1,omega_h1)
      d25 = 0.0; d26 = 0.0;
      # 3rd row ################################################################################################################
      d31 = (2*(mu_over_lambda(lame_coeffs,1, 1)/(nu**2))*kh*(kh - 1) - (1 + 2*mu_over_lambda(lame_coeffs,1, 1))*(omega_h1/kappa1)**2)*jv(kh,omega_h1*nu/kappa1) + 2*(omega_h1/(kappa1*nu))*mu_over_lambda(lame_coeffs,1, 1)*jv(kh+1,omega_h1*nu/kappa1)
      d32 = (2*(mu_over_lambda(lame_coeffs,1, 1)/(nu**2))*kh*(kh - 1) - (1 + 2*mu_over_lambda(lame_coeffs,1, 1))*(kh/kappa1)**2)*yv(kh,omega_h1*nu/kappa1) + 2*(omega_h1/(kappa1*nu))*mu_over_lambda(lame_coeffs,1, 1)*yv(kh+1,omega_h1*nu/kappa1)
      d33 = 2*(mu_over_lambda(lame_coeffs,1, 1)/nu)*1j*kh*(((kh - 1)/nu)*jv(kh,omega_h1*nu) - omega_h1*jv(kh+1,omega_h1*nu))
      d34 = 2*(mu_over_lambda(lame_coeffs,1, 1)/nu)*1j*kh*(((kh - 1)/nu)*yv(kh,omega_h1*nu) - omega_h1*yv(kh+1,omega_h1*nu))
      d35 = -((2*(mu_over_lambda(lame_coeffs,2, 1)/nu**2)*kh*(kh - 1) - (lambda_over_lambda(lame_coeffs,2,1) + 2*mu_over_lambda(lame_coeffs,2, 1))*(omega_h2/kappa2)**2)*jv(kh,omega_h2*nu/kappa2) + 2*(omega_h2/(kappa2*nu))*mu_over_lambda(lame_coeffs,2, 1)*jv(kh+1,omega_h2*nu/kappa2))
      d36 = -2*(mu_over_lambda(lame_coeffs,2, 1)/nu)*1j*kh*(((kh - 1)/nu)*jv(kh,omega_h2*nu) - omega_h2*jv(kh+1,omega_h2*nu))
      # 4th row ################################################################################################################
      d41 = 2*1j*(kh/nu)*(((kh -1)/nu)*jv(kh,omega_h1*nu/kappa1) - (omega_h1/kappa1)*jv(kh+1,omega_h1*nu/kappa1))
      d42 = 2*1j*(kh/nu)*(((kh -1)/nu)*yv(kh,omega_h1*nu/kappa1) - (omega_h1/kappa1)*yv(kh+1,omega_h1*nu/kappa1))
      d43 = (omega_h1**2 - (kh - 1)*(2*kh/(nu**2)))*jv(kh,omega_h1*nu) - (2*omega_h1/nu)*jv(kh+1,omega_h1*nu)
      d44 = (omega_h1**2 - (kh - 1)*(2*kh/(nu**2)))*yv(kh,omega_h1*nu) - (2*omega_h1/nu)*yv(kh+1,omega_h1*nu)
      d45 = 0.0; d46 = 0.0;
      # 5th row ################################################################################################################
      d51 = 0.0; d52 = 0.0; d53 = 0.0; d54 = 0.0;
      d55 = 2*1j*(kh/nu)*(((kh - 1)/nu)*jv(kh,omega_h2*nu/kappa2) - (omega_h2/kappa2)*jv(kh+1,omega_h2*nu/kappa2))
      d56 = (omega_h2**2 - (kh - 1)*(2*kh/(nu**2)))*jv(kh,omega_h2*nu) - 2*(omega_h2/nu)*jv(kh+1,omega_h2*nu)
      # 6th row ################################################################################################################
      d61 = kh*jv(kh,omega_h1*nu/kappa1) - (omega_h1*nu/kappa1)*jv(kh+1,omega_h1*nu/kappa1)
      d62 = kh*yv(kh,omega_h1*nu/kappa1) - (omega_h1*nu/kappa1)*yv(kh+1,omega_h1*nu/kappa1)
      d63 = 1j*kh*jv(kh,omega_h1*nu)
      d64 = 1j*kh*yv(kh,omega_h1*nu)
      d65 = (omega_h2*nu/kappa2)*jv(kh+1,omega_h2*nu/kappa2) - kh*jv(kh,omega_h2*nu/kappa2)
      d66 = -1j*kh*jv(kh,omega_h2*nu)
      # The dij matrix #########################################################################################################
      dij = np.array([
                      [d11 ,d12, d13, d14, d15, d16],
                      [d21, d22, d23, d24, d25, d26],
                      [d31, d32, d33, d34, d35, d36],
                      [d41, d42, d43, d44, d45, d46],
                      [d51, d52, d53, d54, d55, d56],
                      [d61, d62, d63, d64, d65, d66],
                    ])
      return dij

  def D_det(omega):
      '''Calculate the determinant of matrix '''
      return np.linalg.det(Valle_matrix(omega, kp, cL, cT, rho, eta, R))
    # Define constants and range of calculations
  fstop = 1.0*1e6 # Ending frequency in MHz
  dp = 1.0
  wstart = 0.0001*1e6;
  dw = 0.0001*1e6
  kpstart = 0
  kpstop = order # n values 3

  omegas = np.arange(wstart,2*np.pi*fstop,dw)
  kps = np.arange(kpstart,kpstop,dp)

  number_of_roots = modes # l values #10

  roots = []
  KP = []
  for kp in kps: #kp
      print(kp)
      print('mode', 'n', 'freq (Hz)', 'ka')
      rc = 0
      for omega in omegas:
          if kp < 1:
              NR = number_of_roots #- 1 #
              if rc < NR: #number_of_roots:
                  if (np.sign(D_det(omega)) > np.sign(D_det(omega+dw))) or (np.sign(D_det(omega)) < np.sign(D_det(omega+dw))):
                      #print(omega)\
                      root_val = root_scalar(D_det, bracket = [omega,omega+dw], method = 'bisect', xtol = 1e-7)# root_val.root
                      # if rc == 0:
                      #    roots.append(0)
                      #    KP.append(kp)
                      if root_val != None:
                          roots.append(root_val.root)
                          KP.append(kp)
                          print(''+str(int(kp))+'_S_'+str(rc)+'', kp, root_val.root/(2*np.pi), root_val.root*R/cT[1])
                          rc = rc + 1
                          # if rc == 1:
                          #     wstart = root_val.root
                          #     omegas = np.arange(wstart,2*np.pi*fstop,dw)
              elif rc == NR:
                  break
          elif kp >= 1:
              NR = number_of_roots #
              if rc < NR: #number_of_roots:
                  if (np.sign(D_det(omega)) > np.sign(D_det(omega+dw))) or (np.sign(D_det(omega)) < np.sign(D_det(omega+dw))):
                      #print(omega)\
                      root_val = root_scalar(D_det, bracket = [omega,omega+dw], method = 'bisect', xtol = 1e-7)# root_val.root
                      if root_val != None:
                          roots.append(root_val.root)
                          KP.append(kp)
                          print(''+str(int(kp))+'_S_'+str(rc)+'', kp, root_val.root/(2*np.pi), root_val.root*R/cT[1])
                          rc = rc + 1
                          if rc == 1:
                              wstart = root_val.root
                              omegas = np.arange(wstart,2*np.pi*fstop,dw)
              elif rc == NR:
                  break
  return roots, KP

# Calculate the fundamental eigenmodes of Valle 1999

> Valle, C., Qu, J., & Jacobs, L. J. (1999). Guided circumferential waves in layered cylinders. International Journal of Engineering Science, 37(11), 1369-1387. doi: https://doi.org/10.1016/S0020-7225(98)00133-5 [CAUTION: Equations in this paper and her thesis differ]

In [11]:
#@title Calculate fundamental modes
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create input widgets
cL_widget = widgets.Text(value="1000, 1000", description="cL:")
cT_widget = widgets.Text(value="36, 36", description="cT:")
rho_widget = widgets.Text(value="1010, 1010", description="rho:")
eta_widget = widgets.FloatSlider(value=0.01, min=0.01, max=1.0, step=0.01, description="eta:")
R_widget = widgets.FloatText(value=0.02627, description="R:")
order_widget = widgets.FloatText(value=3, description="order:")
modes_widget = widgets.FloatText(value=10, description="modes:")

# Create button to trigger calculation
button = widgets.Button(description="Calculate")

# Create output widget
output = widgets.Output()

# Define calculation function
def calculate(_):
    with output:
        clear_output()
        cL_values = [float(x) for x in cL_widget.value.split(",")]
        cT_values = [float(x) for x in cT_widget.value.split(",")]
        rho_values = [float(x) for x in rho_widget.value.split(",")]
        eta_value = eta_widget.value
        R_value = R_widget.value
        order_value = order_widget.value
        modes_value = modes_widget.value
        roots, KP = Valle_Modes(cL_values, cT_values, rho_values, eta_value, R_value, order_value, modes_value)
    return roots, KP
# Attach calculation function to button click
button.on_click(calculate)

# Display widgets
display(order_widget, modes_widget, cL_widget, cT_widget, rho_widget, eta_widget, R_widget, button, output)

roots, KP = calculate(button)

FloatText(value=3.0, description='order:')

FloatText(value=10.0, description='modes:')

Text(value='1000, 1000', description='cL:')

Text(value='36, 36', description='cT:')

Text(value='1010, 1010', description='rho:')

FloatSlider(value=0.01, description='eta:', max=1.0, min=0.01, step=0.01)

FloatText(value=0.02627, description='R:')

Button(description='Calculate', style=ButtonStyle())

Output()